In [1]:
import pyvisa as pyv
from typing import List
import csv


class WaveformGenerator:
    def __init__(self, resource_string: str):
        self.rm = pyv.ResourceManager()
        self.device = self.rm.open_resource(resource_string)
        response = self.device.query('*IDN?')
        print("Instrument ID:", response)


    def set_waveform_type(self, channel: int, waveform_type: str) -> None:
        command = f'C{channel}:BSWV WVTP,{waveform_type}'
        self.device.write(command)


    def set_frequency(self, channel: int, frequency: float) -> None:
        command = f'C{channel}:BSWV FRQ,{frequency}'
        self.device.write(command)


    def set_amplitude(self, channel: int, amplitude: float) -> None:
        command = f'C{channel}:BSWV AMP,{amplitude}'
        self.device.write(command)


    def set_phase(self, channel: int, phase: float) -> None:
        command = f'C{channel}:BSWV PHSE,{phase}'
        self.device.write(command)


    def start_waveform(self, channel: int) -> None:
        self.device.write(f'C{channel}:OUTP ON')


    def stop_waveform(self, channel: int) -> None:
        self.device.write(f'C{channel}:OUTP OFF')


    def upload_waveform_data(self, channel: int, data: List[float], ip_addr: str) -> None:
        data_str = ','.join(map(str, data))
        command = f'DATA:DATA VOLATILE,{data_str}'
        self.device.write(command)
        self.device.write(f'MEM:STORE VOLATILE,"{ip_addr}"')


    def generate_waveform_data(self, data: List[float]) -> None:
        with open('waveform_data.csv', 'w') as file:
            for value in data:
                file.write(f'{value}\n')


def waveform_config(ip_string: str = None):
    if ip_string is None:
        type_of_connection = input("Enter connection type (i.e., IP, USB, etc.): ").upper()
       
        if type_of_connection == "IP":
            type_of_connection = "TCPIP"
        if type_of_connection == "GPIB":
            type_of_connection = "GPIB0"
        elif type_of_connection == "USB":
            type_of_connection = "USB"
        else:
            print("Invalid connection type. Please enter 'IP', 'USB', or 'GPIB'.")
            return


        ip_address = input("Enter IP or GPIB address: ")
        ip_string = f"{type_of_connection}::{ip_address}::INSTR"


    generator = WaveformGenerator(ip_string)


    while True:
        try:
            ##channel = int(input("Enter the channel number (e.g., 1 or 2): "))
            ##if channel not in [1, 2]:
                ##print("Invalid channel number. Please enter 1 or 2.")
                ##continue


            waveform_type = input("Enter a waveform type (i.e sine, cos, square): ")
            generator.set_waveform_type(waveform_type)


            frequency = float(input("Set frequency in Hz: "))
            generator.set_frequency(frequency)


            amplitude = float(input("Set amplitude: "))
            generator.set_amplitude(amplitude)


            print("Starting waveform")
            generator.start_waveform(1)


            upload = input("Do you want to upload custom waveform data? (yes/no): ").lower()
            if upload == "yes":
                csv_filename = input("Enter the filename of the CSV file (e.g., waveform_data.csv): ")
                try:
                    with open(csv_filename, 'r') as csvfile:
                        reader = csv.reader(csvfile)
                        next(reader)  # Skip the header
                        data_list = [float(row[0]) for row in reader]
                        generator.upload_waveform_data(data_list, ip_string)
                        generator.generate_waveform_data(data_list)
                        print(f"Custom waveform data from {csv_filename} uploaded successfully.")
                except FileNotFoundError:
                    print(f"File {csv_filename} not found.")
                except Exception as e:
                    print(f"An error occurred: {e}")


            stop = input("Do you want to stop the waveform output? (yes/no, this exits both): ").lower()
            if stop == "yes":
                generator.stop_waveform(1)
                generator.stop_waveform(2)
                break


            restart = input("Do you want to reconfigure, or use another channel? (yes/no): ").lower()
            if restart != "yes":
                break


        except ValueError:
            print("Invalid input. Please enter the correct values.")


if __name__ == "__main__":
    waveform_config()


Instrument ID: KEITHLEY INSTRUMENTS INC.,MODEL 6221,4483467,D04  /700x 

Starting waveform
